# vLLM警告抑制と最適化ノートブック

このノートブックでは、vLLMの使用時に表示される様々な警告を抑制し、パフォーマンスを最適化する方法を示します。

In [ ]:
# 必要なライブラリをインポート
import os
import warnings
import logging
import torch
from vllm import LLM, SamplingParams
from transformers import AutoConfig

## 1. 警告抑制設定

vLLMの警告を抑制するための設定を行います。

In [ ]:
def setup_warning_suppression():
    """vLLM関連の警告を抑制する設定"""
    
    # 1. 環境変数による警告抑制
    os.environ['VLLM_LOGGING_LEVEL'] = 'ERROR'  # ログレベルをERRORに設定
    os.environ['VLLM_ALLOW_LONG_MAX_MODEL_LEN'] = '1'  # max_model_len制限を緩和
    os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'  # マルチプロセス方法を明示
    os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # トークナイザー並列処理警告を抑制
    
    # 2. Pythonの警告フィルター設定
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', message='.*safetensors.*')
    warnings.filterwarnings('ignore', message='.*max_num_batched_tokens.*')
    warnings.filterwarnings('ignore', message='.*pin_memory.*')
    warnings.filterwarnings('ignore', message='.*FlashInfer.*')
    
    # 3. ログレベル調整
    logging.getLogger('vllm').setLevel(logging.ERROR)
    logging.getLogger('transformers').setLevel(logging.ERROR)
    
    print("✅ 警告抑制設定完了")

# 警告抑制設定を実行
setup_warning_suppression()

## 2. モデル設定の最適化

モデル設定を確認し、最適なパラメータを設定します。

In [ ]:
def check_model_config_optimized(model_name):
    """最適化されたモデル設定確認"""
    try:
        config = AutoConfig.from_pretrained(model_name)
        max_pos = getattr(config, 'max_position_embeddings', None)
        model_max_len = getattr(config, 'max_length', None)
        
        # 警告を抑制して情報のみ表示
        print(f"📋 Model: {model_name}")
        if max_pos:
            print(f"   max_position_embeddings: {max_pos}")
        if model_max_len:
            print(f"   model_max_length: {model_max_len}")
        
        return max_pos
    except Exception as e:
        print(f"⚠️ 設定確認エラー: {e}")
        return None

# テスト用モデルの設定を確認
check_model_config_optimized("microsoft/DialoGPT-medium")

## 3. 最適化されたLLMインスタンス作成

警告を最小限に抑えたLLMインスタンスを作成します。

In [ ]:
def create_optimized_llm(model_name, max_len=512, gpu_util=0.7):
    """警告を最小化したvLLMインスタンス作成"""
    
    # モデル設定確認
    max_pos = check_model_config_optimized(model_name)
    if max_pos and max_len > max_pos:
        max_len = min(max_len, max_pos)
        print(f"🔧 max_model_lenを{max_len}に調整")
    
    # 最適化されたLLM設定
    llm_config = {
        'model': model_name,
        'trust_remote_code': True,
        'max_model_len': max_len,
        'max_num_seqs': 1,
        'gpu_memory_utilization': gpu_util,
        'enforce_eager': True,  # CUDAグラフ無効化（警告回避）
        'disable_log_stats': True,  # 統計ログ無効化
        'max_num_batched_tokens': max_len,  # バッチトークン数を調整
        'disable_custom_all_reduce': True,  # カスタム削減無効化
        'use_v2_block_manager': False,  # V2ブロックマネージャー無効化
    }
    
    # WSL環境での最適化
    if 'microsoft' in os.uname().release.lower():
        llm_config['pin_memory'] = False
        print("🐧 WSL環境を検出、pin_memory=Falseに設定")
    
    return LLM(**llm_config)

## 4. 最適化されたvLLMテスト

警告を抑制した状態でvLLMをテストします。

In [ ]:
def test_vllm_optimized(model_name, max_len=512, gpu_util=0.7):
    """最適化されたvLLMテスト"""
    print(f"\n🚀 === 最適化テスト: {model_name} ===")
    
    try:
        # 最適化されたLLM作成
        llm = create_optimized_llm(model_name, max_len, gpu_util)
        print(f"✅ {model_name} ロード成功（警告抑制済み）")
        
        # テキスト生成テスト
        sampling_params = SamplingParams(
            temperature=0.2,
            max_tokens=32,
            top_p=0.9
        )
        
        outputs = llm.generate(["Hello, how are you?"], sampling_params)
        result = outputs[0].outputs[0].text
        print(f"✅ テキスト生成成功: {result.strip()}")
        
        # メモリクリーンアップ
        del llm
        torch.cuda.empty_cache()
        
        return True
        
    except Exception as e:
        print(f"❌ {model_name} テスト失敗: {e}")
        torch.cuda.empty_cache()
        return False

# 軽量モデルでテスト
test_vllm_optimized("gpt2", max_len=512, gpu_util=0.6)

## 5. FlashInfer警告の解決

FlashInferに関する警告を解決するためのインストール手順を示します。

In [ ]:
def install_flashinfer():
    """FlashInfer警告を解決するためのインストール手順"""
    print("\n🔧 === FlashInfer警告解決方法 ===")
    print("以下のコマンドでFlashInferをインストールしてください：")
    print("pip install flashinfer -f https://flashinfer.ai/whl/cu121/torch2.4/")
    print("または")
    print("pip install flashinfer")
    print("\n注意: CUDAバージョンとPyTorchバージョンに適合するものを選択してください")

# FlashInferインストール案内を表示
install_flashinfer()

## 6. 複数モデルでの最適化テスト

複数のモデルで最適化設定をテストします。

In [ ]:
# GPU情報表示
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🖥️ GPU Memory: {gpu_memory:.1f} GB")
else:
    print("❌ CUDA not available")

# テストモデル一覧（軽量順）
test_models = [
    ("gpt2", 512, 0.6),
    ("microsoft/DialoGPT-small", 512, 0.7),
    ("microsoft/DialoGPT-medium", 1024, 0.7),
]

print("🎯 === 複数モデルでの最適化テスト ===")
success_count = 0

for model_name, max_len, gpu_util in test_models:
    if test_vllm_optimized(model_name, max_len, gpu_util):
        success_count += 1
    else:
        print(f"⚠️ {model_name}をスキップして次のモデルをテスト")

print(f"\n🎉 === テスト結果: {success_count}/{len(test_models)} 成功 ===")